# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233623217617                   -0.50    7.0
  2   -7.250260843562       -1.78       -1.41    1.0
  3   -7.251274137483       -2.99       -2.02    4.0
  4   -7.251232267688   +   -4.38       -2.12    4.0
  5   -7.251320024052       -4.06       -2.53    3.0
  6   -7.251335789820       -4.80       -2.90    3.0
  7   -7.251338784102       -5.52       -4.07    2.0
  8   -7.251338791988       -8.10       -4.15    5.0
  9   -7.251338798423       -8.19       -4.84    3.0
 10   -7.251338798686       -9.58       -5.41    2.0
 11   -7.251338798701      -10.82       -5.79    3.0
 12   -7.251338798704      -11.48       -6.60    3.0
 13   -7.251338798705      -13.15       -7.08    3.0
 14   -7.251338798705      -13.94       -7.56    3.0
 15   -7.251338798705   +  -14.27       -8.17    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06059833483065516
[ Info: Arnoldi iteration step 2: normres = 0.2989600540439532
[ Info: Arnoldi iteration step 3: normres = 0.7740876812762891
[ Info: Arnoldi iteration step 4: normres = 0.5496951033249264
[ Info: Arnoldi iteration step 5: normres = 0.5314003047751292
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.43e-01, 6.07e-02, 4.52e-01, 2.31e-01, 1.81e-02)
[ Info: Arnoldi iteration step 6: normres = 0.256846161543604
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.00e-02, 6.98e-02, 2.05e-01, 1.07e-01, 4.13e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07220463474022804
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.04e-03, 5.17e-03, 1.30e-02, 3.73e-02, 5.50e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08903289345683386
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.93e-05